In [ ]:
# import necessary packages

import os
import torch
import random
import pickle
import tables
import numpy as np
import torch.nn as nn
from torch import cat
from tqdm import tqdm
from torch.fft import *
import torchkbnufft as tkbn
from einops import rearrange
from scipy.io import savemat
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
# setup seed
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
setup_seed(12345)
# set num threads
torch.set_num_threads(6)    # too many threads slows training

In [ ]:
# load spiral trajectory - see gen_spiral_grad.m

imsize = [96, 96]

trajpath = 'data/spiral/3x'
traj = tables.open_file(os.path.join(trajpath, 'traj.mat')).root.traj

nimgs = traj.real.shape[0]

ktrajs = []
for i in range(nimgs):
    kx, ky = traj.real[i], traj.imag[i]
    kx = torch.from_numpy(kx)
    ky = torch.from_numpy(ky)
    kmax = torch.sqrt(kx**2+ky**2).max()
    kx, ky = kx / kmax * torch.pi, ky / kmax * torch.pi # normalize to (-pi, pi) for torchbknufft
    ktrajs.append(torch.stack([kx,ky]))
    plt.plot(kx, ky)
    plt.show()

In [ ]:
# define utility functions

def ifft2c(k):
    return ifftshift(ifft2(fftshift(k)))

def fft2c(x):
    return fftshift(fft2(ifftshift(x)))

def view_as_complex(x):
    x = torch.chunk(x, chunks=2, dim=1)
    x = x[0] + 1j * x[1]
    return x

def view_as_real(x):
    x = torch.cat([x.real, x.imag], dim=1).float()
    return x

In [ ]:
# define dataset

class dataset(Dataset):
    def __init__(self, datapath, ktrajs, imsize, nimgs):
        self.nimgs  = nimgs
        self.imsize = imsize
        self.ktrajs = torch.stack(ktrajs).cuda()
        # contruct nufft object
        self.nufft = tkbn.KbNufft(imsize).cuda()
        self.adj = tkbn.KbNufftAdjoint(imsize).cuda()
        self.dcf = tkbn.calc_density_compensation_function(self.ktrajs, imsize).cuda()
        # load all data
        names = set([os.path.splitext(name)[0] for name in os.listdir(datapath)])
        self.data_dict = {}
        for name in tqdm(names):
            self.data_dict[name] = {}
            data = torch.from_numpy(np.load(os.path.join(datapath, f'{name}.npy')))
            with open(os.path.join(datapath, f'{name}.pkl'), 'rb') as f:
                offsets = pickle.load(f)
            # sort out M0 images and functional images
            m0imgs, funimgs = [], []
            for offset, img in zip(offsets, data):
                if int(offset[0]) >= 100:
                    img = ifft2c(img)
                    m0imgs.append(img)
                else:
                    funimgs.append(img)
            self.data_dict[name]['m0'] = torch.mean(torch.stack(m0imgs), 0, keepdim=True)
            self.data_dict[name]['kf'] = torch.stack(funimgs)
            self.data_dict[name]['ku'] = {}
            self.data_dict[name]['xu'] = {}
            # preprocess all the data
            self.data_dict[name]['xf'] = ifft2c(self.data_dict[name]['kf'])
            for index in range(len(self.data_dict[name]['xf']) - self.nimgs + 1):
                xf = self.data_dict[name]['xf'][index:index+self.nimgs].unsqueeze(1).cuda()
                self.data_dict[name]['ku'][index] = self.nufft(xf, self.ktrajs).cpu()
                ku = self.data_dict[name]['ku'][index].cuda()
                self.data_dict[name]['xu'][index] = self.adj(ku*self.dcf, self.ktrajs).squeeze(1).cpu()
            del xf, ku
    def __len__(self):
        count = 0
        for key in self.data_dict.keys():
            count += len(self.data_dict[key]['xf']) - self.nimgs + 1
        return count
    def __getitem__(self, index):
        # index corresponds which sample
        for key in self.data_dict.keys():
            nseq = len(self.data_dict[key]['xf']) - self.nimgs + 1
            if index - nseq < 0:
                break
            index -= nseq
        # fetch data
        m0 = self.data_dict[key]['m0']
        # ragard nimgs as a batch
        xf = self.data_dict[key]['xf'][index:index+self.nimgs]
        ku = self.data_dict[key]['ku'][index]
        xu = self.data_dict[key]['xu'][index]/(self.imsize[0]*self.imsize[1])
        return xu, xf, ku, m0

In [ ]:
# define variation network

class Attention(nn.Module):
    def __init__(self, dim, dim_head=64, heads=8):
        super().__init__()
        self.num_heads = heads
        self.dim_head = dim_head
        self.to_q = nn.Linear(dim, dim_head * heads, bias=False)
        self.to_k = nn.Linear(dim, dim_head * heads, bias=False)
        self.to_v = nn.Linear(dim, dim_head * heads, bias=False)
        self.rescale = nn.Parameter(torch.ones(heads, 1, 1))
        self.proj = nn.Linear(dim_head * heads, dim, bias=True)
        self.pos_emb = nn.Conv2d(dim, dim, 3, 1, 1, bias=False, groups=dim)
        self.dim = dim
    def forward(self, x_in):
        b, c, h, w = x_in.shape
        x = x_in.permute(0, 2, 3, 1).reshape(b,h*w,c)
        # b, hw, hd
        q_inp = self.to_q(x)
        k_inp = self.to_k(x)
        v_inp = self.to_v(x)
        # b, h, hw, d
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.num_heads), (q_inp, k_inp, v_inp))
        # b, h, d, hw
        q = q.transpose(-2, -1)
        k = k.transpose(-2, -1)
        v = v.transpose(-2, -1)
        q = F.normalize(q, dim=-1, p=2)
        k = F.normalize(k, dim=-1, p=2)
        # attn: b, h, d, d
        attn = (k @ q.transpose(-2, -1))
        attn = attn * self.rescale
        attn = attn.softmax(dim=-1)
        # x: b, h, d, hw
        x = attn @ v
        x = x.permute(0, 3, 1, 2)
        x = x.reshape(b, h * w, self.num_heads * self.dim_head)
        # out: b, c, h, w
        out_c = self.proj(x).view(b, h, w, c).permute(0, 3, 1, 2)
        out_p = self.pos_emb(x_in)
        out = out_c + out_p
        return out

class FeedForward(nn.Module):
    def __init__(self, dim, mult=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(dim, dim * mult, 1, 1, bias=False),
            nn.GELU(),
            nn.Conv2d(dim * mult, dim * mult, 3, 1, 1, bias=False, groups=dim * mult),
            nn.GELU(),
            nn.Conv2d(dim * mult, dim, 1, 1, bias=False),
        )
    def forward(self, x):
        return self.net(x)

class Transformer(nn.Module):
    def __init__(self, idim, hdim, odim, dim_head=64, heads=8, num_blocks=2):
        super().__init__()
        self.convin = nn.Sequential(nn.Conv2d(idim, hdim, 3, 1, 1, bias=False), nn.GELU())
        self.blocks = nn.ModuleList([])
        for _ in range(num_blocks):
            self.blocks.append(
                nn.ModuleList([
                    Attention(dim=hdim, dim_head=dim_head, heads=heads),
                    FeedForward(dim=hdim)
                ])
            )
        self.convout = nn.Conv2d(hdim, odim, 1, 1, 0)
    def forward(self, x, m0):
        m0 = view_as_real(m0)
        x  = view_as_real(x)
        x  = torch.cat([x,m0],1)
        x  = self.convin(x)
        for (attn, ff) in self.blocks:
            x = attn(x) + x
            x = ff(x) + x
        out = self.convout(x)
        out = view_as_complex(out)
        return out

class vn(nn.Module):
    def __init__(self, idim, hdim, odim, niter, imsize, ktrajs, parallsize):
        super(vn, self).__init__()
        self.parallsize = parallsize
        self.iters = nn.ModuleList()
        for i in range(niter):
            self.iters.append(Transformer(idim, hdim, odim))
        # contruct nufft object
        self.ktrajs = torch.stack(ktrajs).repeat([parallsize,1,1]).cuda()
        self.nufft = tkbn.KbNufft(imsize).cuda()
        self.adj = tkbn.KbNufftAdjoint(imsize).cuda()
        self.dcf = tkbn.calc_density_compensation_function(self.ktrajs, imsize).cuda()
    def forward(self, x, ku, m0):
        batchsize = x.shape[0]
        ind = slice(batchsize*len(self.ktrajs)//self.parallsize)
        ku = rearrange(ku, 'b c h w -> (b c) h w')
        for iter, layer in enumerate(self.iters):
            x = rearrange(x, 'b c h w -> (b c) 1 h w')
            kres = self.nufft(x, self.ktrajs[ind]) - ku
            xdc = self.adj(kres*self.dcf[ind], self.ktrajs[ind])
            xdc = rearrange(xdc, '(b c) 1 h w -> b c h w', b = batchsize)
            x = rearrange(x, '(b c) 1 h w -> b c h w', b = batchsize)
            xdc = xdc / abs(xdc).max()
            x = x + layer(x,m0) - xdc
        return x

In [ ]:
# define learning rate scheduler

class PolyScheduler(LambdaLR):
    def __init__(self, optimizer, t_total, exponent=0.9, last_epoch=-1):
        self.t_total = t_total
        self.exponent = exponent
        super(PolyScheduler, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        return (1 - step / self.t_total)**self.exponent

In [ ]:
# run training

batchsize = 8
trainset = dataset('data/processed/trains', ktrajs=ktrajs, imsize=imsize, nimgs=nimgs)
trainloader = DataLoader(trainset, batchsize, shuffle=True, drop_last=True)

# take a snapshot of xu, xf, and m0
xu, xf, ku, m0 = trainset.__getitem__(0)
print(xu.shape, xf.shape, ku.shape, m0.shape)
for i in range(nimgs):
    plt.subplot(2, nimgs, i+1)
    plt.imshow(abs(xu)[i].cpu().numpy(), cmap='gray')
    plt.subplot(2, nimgs, nimgs+i+1)
    plt.imshow(abs(xf)[i].cpu().numpy(), cmap='gray')
plt.show()

# hyper-parameters
lr, weight_decay, epochs = 1e-4, 1e-4, 60

net = vn(idim=2*(1+nimgs), hdim=16, odim=2*nimgs, niter=6, ktrajs=ktrajs, imsize=imsize, parallsize=batchsize).cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = PolyScheduler(optimizer, t_total=epochs)
criterion = nn.L1Loss()

iter = 0
writer = SummaryWriter()
scaler = torch.cuda.amp.GradScaler()
for epoch in range(epochs):
    for xu, xf, ku, m0 in trainloader:
        xu, xf, ku, m0 = xu.cuda(), xf.cuda(), ku.cuda(), m0.cuda()
        xr = net(xu, ku, m0)
        loss = criterion(xr, xf)
        # backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(),1.0)
        optimizer.step()
        # log
        writer.add_scalar('Loss/train', loss.item(), iter)
        iter = iter + 1
    scheduler.step()

torch.save(net.state_dict(), 'variables/motr_spiral/motr.pth')

# take a snapshot of xr and xf
for i in range(nimgs):
    plt.subplot(2, nimgs, i+1)
    plt.imshow(abs(xr)[0][i].detach().cpu().numpy())
    plt.subplot(2, nimgs, nimgs+i+1)
    plt.imshow(abs(xf)[0][i].detach().cpu().numpy())
plt.show()